# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 71 new papers today
          27 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/26 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.01382


extracting tarball to tmp_2211.01382...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.01384


extracting tarball to tmp_2211.01384...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.01927


extracting tarball to tmp_2211.01927...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.01935


extracting tarball to tmp_2211.01935...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01474-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01474) | **Peering into the Young Planetary System AB Pic. Atmosphere, Orbit,  Obliquity & Second Planetary Candidate**  |
|| P. Palma-Bifani, et al. -- incl., <mark>M. Langlois</mark>, <mark>C. Desgrange</mark>, <mark>M. Feldt</mark>, <mark>M. Meyer</mark>, <mark>M. Samland</mark>, <mark>Y. Zhang</mark> |
|*Appeared on*| *2022-11-04*|
|*Comments*| *17 pages, 13 Figures, 6 Tables. Accepted for publication in A&A (31 of October)*|
|**Abstract**| We aim to revisit the system AB Pic which has a known companion at the exoplanet/ brown-dwarf boundary. We based this study on a rich set of observations to investigate the companion's orbit and atmosphere. We composed a spectrum of AB Pic b merging archival VLT/SINFONI K-band data, with published spectra at J and H-band (SINFONI) and Lp-band (Magellan-AO), and photometric measurements (HST and Spitzer). We modeled the spectrum with ForMoSA, based on two atmospheric models: ExoREM and BT-SETTL13. We determined the orbital properties of b fitting the astrometric measurements from NaCo (2003 and 2004) and SPHERE (2015). The orbital solutions favor a semi-major axis of $\sim$190au viewed edge-on. With Exo-REM, we derive a T$_{eff}$ of 1700$\pm$50K and surface gravity of 4.5$\pm$0.3dex, consistent with previous works, and we report for the first time a C/O ratio of 0.58$\pm$0.08 ($\sim$solar). The posteriors are sensitive to the wavelength interval and the family of models used. Given the 2.1hr rotation period and our vsin(i) of $\sim$73km/s, we estimate for the first time the true obliquity to be $\sim$45 or $\sim$135deg, indicating a significant misalignment between the planet's spin and orbit orientations. Finally, the existence of a proper motion anomaly between the Hipparcos and Gaia eDR3 compared to our SPHERE detection limits and adapted radial velocity limits indicate the existence of a $\sim$6M$_{Jup}$ inner planet orbiting from 2 to 10au (40-200mas). The possible existence of an inner companion, together with the likely miss-alignment of the spin axis orientation, strongly favor a formation path by gravitational instability or core accretion within a disk closer inside followed by dynamical interactions. Confirmation and characterization of planet c and access to a broader wavelength coverage for planet b will be essential to probe the uncertainties associated with the parameters. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01382-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01382) | **The Mass-Metallicity Relation of Dwarf Galaxies at the Cosmic Noon in  the JWST Era**  |
|| <mark>Mingyu Li</mark>, et al. -- incl., <mark>Xiaojing Lin</mark>, <mark>Zihao Li</mark>, <mark>Shiwu Zhang</mark> |
|*Appeared on*| *2022-11-04*|
|*Comments*| *16 pages, 4 figures, 1 table, submitted to AAS Journal; welcome comments*|
|**Abstract**| We present the mass-metallicity relation (MZR) at $z=2-3$ in the stellar mass range of $M_\star\approx 10^{6.5}-10^{9.5}M_\odot$ using 55 dwarf galaxies in the Abell 2744 and SMACS J0723-3732 galaxy cluster fields. These dwarf galaxies are identified and confirmed by deep JWST/NIRISS imaging and slitless grism spectroscopic observations. Taking advantage of the gravitational lensing effect, we extend the previous MZR relation at $z=2-3$ to a much lower mass regime by more than 2.5 orders of magnitude compared with previous studies. We find that the MZR has a shallower slope at the low-mass end ($M_\star<10^{9}M_\odot$) compared to that at the high-mass end ($M_\star>10^{9}M_\odot$), with a slope turnover point at around the stellar mass of $10^9 M_\odot$. This implies that dominating feedback processes in dwarf galaxies may be different from that in galaxies with higher mass. From $z=3$ to $z=2$, the metallicity of the dwarf galaxies is enhanced by $\approx0.1$ dex for a given stellar mass, consistent with the mild evolution found in galaxies with higher mass. Further, we confirm the existence of a 3D relation between the gas-phase metallicity, stellar mass, and star formation rate, i.e., fundamental metallicity relation (FMR), in dwarf galaxies at $z=2-3$. Our derived FMR, which has no significant redshift evolution, can be used as a benchmark to understand the origin of the anti-correlation between SFR and metallicity of dwarf galaxies in the high-redshift Universe. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01384-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01384) | **Distinguishing Dark Matter Cusps from Cores using Globular Clusters**  |
|| Shaunak Modak, Shany Danieli, <mark>Jenny E. Greene</mark> |
|*Appeared on*| *2022-11-04*|
|*Comments*| *19 pages, 11 figures. Main results in figures 7 and 8. Submitted to ApJ, comments are welcome!*|
|**Abstract**| Globular Clusters (GCs) provide valuable insight into the properties of their host galaxies' dark matter halos. Using N-body simulations incorporating semianalytic dynamical friction and GC-GC merger prescriptions, we study the evolution of GC radial distributions and mass functions in cuspy and cored dark matter halos. Modeling the dynamics of the GC-rich system in the dwarf galaxy UGC7369, we find that friction-induced inspiral and subsequent mergers of massive GCs can naturally and robustly explain the mass segregation of the GCs and the existence of a nuclear star cluster (NSC). However, the multiple mergers required to form the NSC only take place when the dark matter halo is cuspy. In a cored halo, stalling of the dynamical friction within the core halts the inspiral of the GCs, and so the GC merger rate falls significantly, precluding the formation of an NSC. We therefore argue that the presence of an NSC requires a cusp in UGC7369. More generally, we propose that the presence of an NSC and the corresponding alteration of the GC mass function due to mergers may be used as an indicator of a cuspy halo for galaxies in which we expect NSC formation to be merger-dominated. These observables represent a simple, powerful complement to other inner halo density profile constraint techniques, and should allow for straightforward extension to larger samples. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01897-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01897) | **Demographics of the M-star Multiple Population in the Orion Nebula  Cluster**  |
|| Matthew De Furio, et al. -- incl., <mark>Christopher Liu</mark>, <mark>Michael R. Meyer</mark> |
|*Appeared on*| *2022-11-04*|
|*Comments*| *23 pages, 11 figures, Accepted to ApJ Nov. 3, 2022*|
|**Abstract**| We present updated results constraining multiplicity demographics for the stellar population of the Orion Nebula Cluster (ONC, a high-mass, high-density star-forming region), across primary masses 0.08-0.7M$_{\odot}$. Our study utilizes archival Hubble Space Telescope data obtained with the Advanced Camera for Surveys using multiple filters (GO-10246). Previous multiplicity surveys in low-mass, low-density associations like Taurus identify an excess of companions to low-mass stars roughly twice that of the Galactic field and find the mass ratio distribution consistent with the field. Previously, we found the companion frequency to low-mass stars in the ONC is consistent with the Galactic field over mass ratios=0.6-1.0 and projected separations=30-160au, without placing constraints on the mass ratio distribution. In this study, we investigate the companion population of the ONC with a double point-spread function (PSF) fitting algorithm sensitive to separations larger than 10au (0.025") using empirical PSF models. We identified 44 companions (14 new), and with a Bayesian analysis, estimate the companion frequency to low-mass stars in the ONC =0.13$^{+0.05}_{-0.03}$ and the power law fit index to the mass ratio distribution =2.08$^{+1.03}_{-0.85}$ over all mass ratios and projected separations of 10-200au. We find the companion frequency in the ONC is consistent with the Galactic field population, likely from high transient stellar density states, and a probability of 0.002 that it is consistent with that of Taurus. We also find the ONC mass ratio distribution is consistent with the field and Taurus, potentially indicative of its primordial nature, a direct outcome of the star formation process. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01922-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01922) | **Star Formation Variability as a Probe for the Baryon Cycle within  Galaxies**  |
|| Eun-jin Shin, et al. -- incl., <mark>Vadim A. Semenov</mark> |
|*Appeared on*| *2022-11-04*|
|*Comments*| *22 pages including 17 figures. Submitted to ApJ*|
|**Abstract**| We investigate the connection of the regulation of star formation and the cycling of baryons within and in and out of galaxies. We use idealized numerical simulations of Milky Way-mass galaxies, in which we systemically vary the galaxy morphology (bulge-to-total mass ratio) and stellar feedback strength (total eight setups with 80 simulations). By following individual gas parcels through the disk, spiral arms, and massive star-forming clumps, we quantify how gas moves and oscillates through the different phases of the interstellar medium (ISM) and forms stars. We show that the residence time of gas in the dense ISM phase ($\tau_{\rm SF}$), the nature of spiral arms (strength, number), and the clump properties (number, mass function, and young star fraction) depend on both the galaxy morphology and stellar feedback. Based on these results, we quantify signatures of the baryon cycle within galaxies using the temporal and spatial power spectrum density (PSD) of the star formation history (SFH). Stronger stellar feedback leads to more bursty star formation while the correlation timescale of the SFH is longer, because stronger feedback dissolves the dense, star-forming ISM phase, leading to a more homogeneous ISM and a decrease in $\tau_{\rm SF}$. The bulge strength has a similar effect: the deep gravitational potential in a bulge-dominant galaxy imposes a strong shear force that effectively breaks apart gas clumps in the ISM; this subsequently inhibits the fragmentation of cool gas and therefore the star formation in the disk, leading to a decrease in the spatial power on scales of $\sim$ 1 kpc. We conclude that measurements of the temporal and spatial PSD of the SFH can provide constraints on the baryon cycle and the star formation process. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01927-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01927) | **Modeling Solids in Nuclear Astrophysics with Smoothed Particle  Hydrodynamics**  |
|| Irina Sagert, et al. -- incl., <mark>Hyun Lim</mark> |
|*Appeared on*| *2022-11-04*|
|*Comments*| *24 pages, 29 figures*|
|**Abstract**| Smoothed Particle Hydrodynamics (SPH) is a frequently applied tool in computational astrophysics to solve the fluid dynamics equations governing the systems under study. For some problems, for example when involving asteroids and asteroid impacts, the additional inclusion of material strength is necessary in order to accurately describe the dynamics. In compact stars, that is white dwarfs and neutron stars, solid components are also present. Neutron stars have a solid crust which is the strongest material known in nature. However, their dynamical evolution, when modeled via SPH or other computational fluid dynamics codes, is usually described as a purely fluid dynamics problem. Here, we present the first 3D simulations of neutron-star crustal toroidal oscillations including material strength with the Los Alamos National Laboratory SPH code FleCSPH. In the first half of the paper, we present the numerical implementation of solid material modeling together with standard tests. The second half is on the simulation of crustal oscillations in the fundamental toroidal mode. Here, we dedicate a large fraction of the paper to approaches which can suppress numerical noise in the solid. If not minimized, the latter can dominate the crustal motion in the simulations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01935-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01935) | **Discovering a new well: Decaying dark matter with profile likelihoods**  |
|| Emil Brinch Holm, et al. -- incl., <mark>Laura Herold</mark> |
|*Appeared on*| *2022-11-04*|
|*Comments*| *6 pages, 4 figures. Comments welcome!*|
|**Abstract**| A large number of studies, all using Bayesian parameter inference from Markov Chain Monte Carlo methods, have constrained the presence of a decaying dark matter component. All such studies find a strong preference for either very long-lived or very short-lived dark matter. However, in this letter, we demonstrate that this preference is due to parameter volume effects that drive the model towards the standard $\Lambda$CDM model, which is known to provide a good fit to most observational data. Using profile likelihoods, which are free from volume effects, we instead find that the best-fitting parameters are associated with an intermediate regime where around $3 \%$ of cold dark matter decays just prior to recombination. With two additional parameters, the model yields an overall preference over the $\Lambda$CDM model of $\Delta \chi^2 \approx -2.8$ with \textit{Planck} and BAO and $\Delta \chi^2 \approx -7.8$ with the SH0ES $H_0$ measurement, while only slightly alleviating the $H_0$ tension. Ultimately, our results reveal that decaying dark matter is more viable than previously assumed, and illustrate the dangers of relying exclusively on Bayesian parameter inference when analysing extensions to the $\Lambda$CDM model. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01740-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01740) | **Formation and Thermodynamic Evolution of plasmoids in active region jets**  |
|| Sargam M. Mulay, et al. -- incl., <mark>Helen Mason</mark> |
|*Appeared on*| *2022-11-04*|
|*Comments*| *13 pages, 10 figures, Accepted for publication in Monthly Notices of the Royal Astronomical Society*|
|**Abstract**| We have carried out a comprehensive study of the temperature structure of plasmoids, which successively occurred in recurrent active region jets. The multithermal plasmoids were seen to be travelling along the multi-threaded spire as well as at the footpoint region in the EUV/UV images recorded by the Atmospheric Imaging Assembly (AIA). The Differential Emission Measure (DEM) analysis was performed using EUV AIA images, and the high-temperature part of the DEM was constrained by combining X-ray images from the X-ray telescope (XRT/Hinode). We observed a systematic rise and fall in brightness, electron number densities and the peak temperatures of the spire plasmoid during its propagation along the jet. The plasmoids at the footpoint (FPs) (1.0-2.5 MK) and plasmoids at the spire (SPs) (1.0-2.24 MK) were found to have similar peak temperatures, whereas the FPs have higher DEM weighted temperatures (2.2-5.7 MK) than the SPs (1.3-3.0 MK). A lower limit to the electron number densities of plasmoids - SPs (FPs) were obtained that ranged between 3.4-6.1$\times$10$^{8}$ (3.3-5.9$\times$10$^{8}$) cm$^{-3}$ whereas for the spire, it ranged from 2.6-3.2$\times$10$^{8}$ cm$^{-3}$. Our analysis shows that the emission of these plasmoids starts close to the base of the jet(s), where we believe that a strong current interface is formed. This suggests that the blobs are plasmoids induced by a tearing-mode instability. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.01474.md
    + _build/html/tmp_2211.01474/./img_AB_Pic_b/Full_fit_xlog_for_publication_NS.png
    + _build/html/tmp_2211.01474/./img_AB_Pic_b/K_fit_for_publication_NS_all.png
    + _build/html/tmp_2211.01474/./img_AB_Pic_b/J_fit_nocont_for_publication_NS_vel.png
    + _build/html/tmp_2211.01474/./img_AB_Pic_b/detLimits_RVDIPmA.png
    + _build/html/tmp_2211.01474/./img_AB_Pic_b/corner_BTSETTL_paper.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\blue}[1]{\textcolor{blue}{#1}}$
$\newcommand{\teal}[1]{\textcolor{teal}{#1}}$
$\newcommand{\cyan}[1]{\textcolor{cyan}{#1}}$
$\newcommand{\violet}[1]{\textcolor{violet}{#1}}$
$\newcommand{\magenta}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\fgreen}[1]{\textcolor{ForestGreen}{#1}}$
$\newcommand{\lgreen}[1]{\textcolor{LimeGreen}{#1}}$
$\newcommand{\nblue}[1]{\textcolor{NavyBlue}{#1}}$
$\newcommand{\theenumi}{\roman{enumi}}$
$\newcommand{\arraystretch}{1.7}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\red$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\blue$}[1]{\textcolor{blue}{#1}}$
$\newcommand{$\teal$}[1]{\textcolor{teal}{#1}}$
$\newcommand{$\cyan$}[1]{\textcolor{cyan}{#1}}$
$\newcommand{$\violet$}[1]{\textcolor{violet}{#1}}$
$\newcommand{$\magenta$}[1]{\textcolor{magenta}{#1}}$
$\newcommand{$\fgreen$}[1]{\textcolor{ForestGreen}{#1}}$
$\newcommand{$\lgreen$}[1]{\textcolor{LimeGreen}{#1}}$
$\newcommand{$\nblue$}[1]{\textcolor{NavyBlue}{#1}}$
$\newcommand{$\theenumi$}{\roman{enumi}}$
$\newcommand{$\arraystretch$}{1.7}$</div>



<div id="title">

# Peering into the Young Planetary System AB Pic:  Second Planetary Candidate

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.01474-b31b1b.svg)](https://arxiv.org/abs/2211.01474)<mark>Appeared on: 2022-11-04</mark> - _17 pages, 13 Figures, 6 Tables. Accepted for publication in A&A (31 of October)_

</div>
<div id="authors">

P. Palma-Bifani, et al. -- incl., <mark><mark>M. Langlois</mark></mark>, <mark><mark>C. Desgrange</mark></mark>, <mark><mark>M. Feldt</mark></mark>, <mark><mark>M. Meyer</mark></mark>, <mark><mark>M. Samland</mark></mark>, <mark><mark>Y. Zhang</mark></mark>

</div>
<div id="abstract">

**Abstract:** We aim to revisit the system AB Pic which has a known companion at the exoplanet/ brown-dwarf boundary. We based this study on a rich set of observations to investigate the companion's orbit and atmosphere. We composed a spectrum of AB Pic b merging archival VLT/SINFONI K-band data, with published spectra at J and H-band (SINFONI) and Lp-band (Magellan-AO), and photometric measurements (HST and Spitzer). We modeled the spectrum with ForMoSA, based on two atmospheric models: ExoREM and BT-SETTL13. We determined the orbital properties of b fitting the astrometric measurements from NaCo (2003 and 2004) and SPHERE (2015). The orbital solutions favor a semi-major axis of $\sim$190au viewed edge-on. With Exo-REM, we derive a T$_{eff}$ of 1700$\pm$50K and surface gravity of 4.5$\pm$0.3dex, consistent with previous works, and we report for the first time a C/O ratio of 0.58$\pm$0.08 ($\sim$solar). The posteriors are sensitive to the wavelength interval and the family of models used. Given the 2.1hr rotation period and our vsin(i) of $\sim$73km/s, we estimate for the first time the true obliquity to be $\sim$45 or $\sim$135deg, indicating a significant misalignment between the planet's spin and orbit orientations. Finally, the existence of a proper motion anomaly between the Hipparcos and Gaia eDR3 compared to our SPHERE detection limits and adapted radial velocity limits indicate the existence of a $\sim$6M$_{Jup}$ inner planet orbiting from 2 to 10au (40-200mas). The possible existence of an inner companion, together with the likely miss-alignment of the spin axis orientation, strongly favor a formation path by gravitational instability or core accretion within a disk closer inside followed by dynamical interactions. Confirmation and characterization of planet c and access to a broader wavelength coverage for planet b will be essential to probe the uncertainties associated with the parameters. 

</div>

<div id="div_fig1">

<img src="tmp_2211.01474/./img_AB_Pic_b/Full_fit_xlog_for_publication_NS.png" alt="Fig10.1" width="33%"/><img src="tmp_2211.01474/./img_AB_Pic_b/K_fit_for_publication_NS_all.png" alt="Fig10.2" width="33%"/><img src="tmp_2211.01474/./img_AB_Pic_b/J_fit_nocont_for_publication_NS_vel.png" alt="Fig10.3" width="33%"/>

**Figure 10. -** Top: Full SED fit for both families of atmospheric models and the residuals (data - model). An offset was applied to the BT-SETTL13 model.
    Middle: Same as the top but for the K-band only.
    Bottom: Same as the top but for the J-band modeled without the continuum. Here, the gray area ($>1.3  \mu$m) stands for the spectral region excluded from the fit.
    Each model is identifiable by the label with the numeration of Tables \ref{ForMoSA1} and \ref{ForMoSA2}. The colors are consistent with the corner plots (Figures \ref{Corner_1} and \ref{Corner_2}) and the Tables. (*Fit*)

</div>
<div id="div_fig2">

<img src="tmp_2211.01474/./img_AB_Pic_b/detLimits_RVDIPmA.png" alt="Fig7" width="100%"/>

**Figure 7. -** Mass detection limits for an inner planet without assuming coplanarity with AB Pic b. The pink curve shows the estimated mass and orbital distance from the \textit{Hipparcos}-\textit{Gaia} proper motion anomaly (PMa), the blue area represents the SPHERE detection limits (DI), and the green area represents the RV detection limits from \citet{Grandjean2020} to give a rough idea of the inner regions that we can exclude. The semi-major axis and mass of AB Pic b is represented by the white point. (*detLimABPicc*)

</div>
<div id="div_fig3">

<img src="tmp_2211.01474/./img_AB_Pic_b/corner_BTSETTL_paper.png" alt="Fig11" width="100%"/>

**Figure 11. -** Corner-plot comparing the posteriors of the models presented on Figure \ref{Fit} for BT-SETTL13 with consistent colors. The black dashed lines represent the adopted values from this work listed in Table \ref{tab:atmosparams}. (*Corner_1*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

81  publications in the last 7 days.
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers